In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
ship_movements = pd.read_csv(r'C:\Users\User\Downloads\vessel_movements_PPT.csv')

In [3]:
ship_movements

,imo,mmsi,vessel_id,vessel_name,date_of_build,vessel_type,group,timestamp,date_only,lon,...,anchorage,terminal,maneuvering_zone,p,vref,sfc_me,sfc_ae,sfc_ab,ael,abl
0,9223253,525119038,692017,PRIMA TANGGUH LVI,2002-04-01,Crude/Oil Products Tanker,Crude & Products Tanker,2023-09-17T00:01:42.000Z,2023-09-17,103.826065,...,PPT Anchorage,NaN,NaN,7860.0,14.5,185.00,230.0,300.0,507.0,365.0
1,9223253,525119038,692017,PRIMA TANGGUH LVI,2002-04-01,Crude/Oil Products Tanker,Crude & Products Tanker,2023-09-17T00:01:44.000Z,2023-09-17,103.826065,...,PPT Anchorage,NaN,NaN,7860.0,14.5,185.00,230.0,300.0,507.0,365.0
2,9223253,525119038,692017,PRIMA TANGGUH LVI,2002-04-01,Crude/Oil Products Tanker,Crude & Products Tanker,2023-09-17T00:07:43.000Z,2023-09-17,103.825690,...,PPT Anchorage,NaN,NaN,7860.0,14.5,185.00,230.0,300.0,507.0,365.0
3,9223253,525119038,692017,PRIMA TANGGUH LVI,2002-04-01,Crude/Oil Products Tanker,Crude & Products Tanker,2023-09-17T00:07:45.000Z,2023-09-17,103.825690,...,PPT Anchorage,NaN,NaN,7860.0,14.5,185.00,230.0,300.0,507.0,365.0
4,9223253,525119038,692017,PRIMA TANGGUH LVI,2002-04-01,Crude/Oil Products Tanker,Crude & Products Tanker,2023-09-17T00:10:42.000Z,2023-09-17,103.825580,...,PPT Anchorage,NaN,NaN,7860.0,14.5,185.00,230.0,300.0,507.0,365.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2822594,9708693,374766000,714547,MSC CLARA,2015-11-01,Container Ship (Fully Cellular),Container Ship,2023-08-21T21:59:17.000Z,2023-08-21,103.761665,...,NaN,NaN,PPT Maneouvering Zone,62500.0,19.0,175.00,220.0,300.0,NaN,NaN
2822595,9708693,374766000,714547,MSC CLARA,2015-11-01,Container Ship (Fully Cellular),Container Ship,2023-08-21T22:02:17.000Z,2023-08-21,103.761665,...,NaN,NaN,PPT Maneouvering Zone,62500.0,19.0,175.00,220.0,300.0,4493.0,908.0
2822596,9449106,636020361,647347,MENDELSSOHN,2012-04-01,Container Ship (Fully Cellular),Container Ship,2023-08-21T21:58:47.000Z,2023-08-21,103.783330,...,NaN,NaN,PPT Maneouvering Zone,31990.0,23.0,174.83,220.0,300.0,NaN,NaN
2822597,9449106,636020361,647347,MENDELSSOHN,2012-04-01,Container Ship (Fully Cellular),Container Ship,2023-08-21T22:01:47.000Z,2023-08-21,103.783330,...,NaN,NaN,PPT Maneouvering Zone,31990.0,23.0,174.83,220.0,300.0,2503.0,436.0


In [4]:
import geopandas as gpd
from shapely.geometry import Point

maneuvering_zone = gpd.read_file(r'C:\Users\User\Downloads\drive-download-20240126T095513Z-001\PPT_maneuveringzone.shp')
port_boundary = gpd.read_file(r'C:\Users\User\Downloads\drive-download-20240126T095513Z-001\PPT_portboundary.shp')
berth_polygon = gpd.read_file(r'C:\Users\User\Downloads\drive-download-20240126T095513Z-001\PPT_berths.shp')
anchorage = gpd.read_file(r'C:\Users\User\Downloads\drive-download-20240126T095513Z-001\PPT_anchorage.shp')

In [5]:
# Function to determine if a point is outside maneuvering zone but inside port fence
def is_outside_maneuvering_zone_inside_port_fence(lon, lat):
    point = Point(lon, lat)
    return point.within(port_boundary.unary_union) and not point.within(maneuvering_zone.unary_union)

def is_inside_maneuvering_zone(lon, lat):
    point = Point(lon, lat)
    return point.within(maneuvering_zone.unary_union)

# Function to determine if a point is within the berth polygon
def is_within_berth(lon, lat):
    point = Point(lon, lat)
    return point.within(berth_polygon.unary_union)

def is_within_anchorage(lon, lat):
    point = Point(lon, lat)
    return point.within(anchorage.unary_union)

def determine_operating_mode(row):
    moving_statuses = {0, 3, 4, 8, 11, 12}
    speed_condition = row['speed'] > 1
    status_condition = row['nav_stat'] in moving_statuses
    location_condition = is_outside_maneuvering_zone_inside_port_fence(row['lon'], row['lat'])
    
    if speed_condition or status_condition:
        if location_condition:
            return "Transit"
        elif is_inside_maneuvering_zone(row['lon'], row['lat']):
            return "Maneuvering"
        else: 
            return "Unknown"
    elif not speed_condition:
        if is_within_berth(row['lon'], row['lat']):
            return "Hotel"
        elif is_within_anchorage(row['lon'], row['lat']):
            return "Anchorage"
        else :
            return "Unknown"
    else:
        return "Unknown"

# Apply the function to create the 'operating_mode' column
ship_movements['operating_mode'] = ship_movements.apply(determine_operating_mode, axis=1)


In [6]:
ship_movements

,imo,mmsi,vessel_id,vessel_name,date_of_build,vessel_type,group,timestamp,date_only,lon,...,terminal,maneuvering_zone,p,vref,sfc_me,sfc_ae,sfc_ab,ael,abl,operating_mode
0,9223253,525119038,692017,PRIMA TANGGUH LVI,2002-04-01,Crude/Oil Products Tanker,Crude & Products Tanker,2023-09-17T00:01:42.000Z,2023-09-17,103.826065,...,NaN,NaN,7860.0,14.5,185.00,230.0,300.0,507.0,365.0,Anchorage
1,9223253,525119038,692017,PRIMA TANGGUH LVI,2002-04-01,Crude/Oil Products Tanker,Crude & Products Tanker,2023-09-17T00:01:44.000Z,2023-09-17,103.826065,...,NaN,NaN,7860.0,14.5,185.00,230.0,300.0,507.0,365.0,Anchorage
2,9223253,525119038,692017,PRIMA TANGGUH LVI,2002-04-01,Crude/Oil Products Tanker,Crude & Products Tanker,2023-09-17T00:07:43.000Z,2023-09-17,103.825690,...,NaN,NaN,7860.0,14.5,185.00,230.0,300.0,507.0,365.0,Anchorage
3,9223253,525119038,692017,PRIMA TANGGUH LVI,2002-04-01,Crude/Oil Products Tanker,Crude & Products Tanker,2023-09-17T00:07:45.000Z,2023-09-17,103.825690,...,NaN,NaN,7860.0,14.5,185.00,230.0,300.0,507.0,365.0,Anchorage
4,9223253,525119038,692017,PRIMA TANGGUH LVI,2002-04-01,Crude/Oil Products Tanker,Crude & Products Tanker,2023-09-17T00:10:42.000Z,2023-09-17,103.825580,...,NaN,NaN,7860.0,14.5,185.00,230.0,300.0,507.0,365.0,Anchorage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2822594,9708693,374766000,714547,MSC CLARA,2015-11-01,Container Ship (Fully Cellular),Container Ship,2023-08-21T21:59:17.000Z,2023-08-21,103.761665,...,NaN,PPT Maneouvering Zone,62500.0,19.0,175.00,220.0,300.0,NaN,NaN,Maneuvering
2822595,9708693,374766000,714547,MSC CLARA,2015-11-01,Container Ship (Fully Cellular),Container Ship,2023-08-21T22:02:17.000Z,2023-08-21,103.761665,...,NaN,PPT Maneouvering Zone,62500.0,19.0,175.00,220.0,300.0,4493.0,908.0,Maneuvering
2822596,9449106,636020361,647347,MENDELSSOHN,2012-04-01,Container Ship (Fully Cellular),Container Ship,2023-08-21T21:58:47.000Z,2023-08-21,103.783330,...,NaN,PPT Maneouvering Zone,31990.0,23.0,174.83,220.0,300.0,NaN,NaN,Maneuvering
2822597,9449106,636020361,647347,MENDELSSOHN,2012-04-01,Container Ship (Fully Cellular),Container Ship,2023-08-21T22:01:47.000Z,2023-08-21,103.783330,...,NaN,PPT Maneouvering Zone,31990.0,23.0,174.83,220.0,300.0,2503.0,436.0,Maneuvering
